#  Banca Trinacria – Data Generator (Parte 2)

**Generatore di dati bancari fittizi ma realistici** 

Progetto portfolio – *Vincenzo Alesi, Data Analyst*

 I dati si basano sui CSV prodotti nella **Parte 1**

---

Questo notebook genera:

- Conti correnti
- Transazioni bancarie
- Prestiti e classificazione NPL


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from pathlib import Path
from typing import List, Dict, Tuple
import warnings

warnings.filterwarnings("ignore")

np.random.seed(42)
random.seed(42)

In [2]:
class Config:
    DATA_DIR = Path(r"C:\Users\Vincenzo\Desktop\Banca\Python\data\csv_output")
    DATA_INIZIO = datetime(2015, 1, 1)
    DATA_FINE = datetime(2024, 12, 31)
    NPL_RATIO = 0.08
    FRAUD_RATE = 0.002

# crea la cartella se non esiste
Config.DATA_DIR.mkdir(parents=True, exist_ok=True)

##  Caricamento dati base
Caricamento dei dataset generati nella Parte 1:
- Clienti
- Filiali
- Dipendenti

In [3]:
def carica_dati_base():
    print("Caricamento dati base...")
    
    df_clienti = pd.read_csv(Config.DATA_DIR / "clienti.csv")
    df_filiali = pd.read_csv(Config.DATA_DIR / "filiali.csv")
    df_dipendenti = pd.read_csv(Config.DATA_DIR / "dipendenti.csv")
    
    print(f"✓ Clienti: {len(df_clienti)} | Filiali: {len(df_filiali)} | Dipendenti: {len(df_dipendenti)}")
    return df_clienti, df_filiali, df_dipendenti

In [4]:
df_clienti, df_filiali, df_dipendenti = carica_dati_base()

Caricamento dati base...
✓ Clienti: 50000 | Filiali: 32 | Dipendenti: 350


##  Generazione conti correnti
- 1–3 conti per cliente
- Saldo iniziale realistico
- Fido basato su credit score

In [5]:
def genera_iban(filiale_id: int) -> str:
    """Genera IBAN italiano fittizio"""
    check_digits = random.randint(10, 99)
    cin = random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    abi = '03062'
    cab = str(filiale_id).zfill(5)
    conto = str(random.randint(0, 999999999999)).zfill(12)
    return f'IT{check_digits}{cin}{abi}{cab}{conto}'


def genera_conti(df_clienti: pd.DataFrame, df_filiali: pd.DataFrame) -> pd.DataFrame:
    """
    Genera conti correnti per i clienti.
    Distribuzione: 1-3 conti per cliente (media 1.5)
    """
    print("\nGenerazione conti correnti...")
    
    conti = []
    conto_id = 1
    
    for _, cliente in df_clienti.iterrows():
        # Numero conti: dipende da segmento
        if cliente['segmento'] == 'Private':
            num_conti = random.choices([1, 2, 3], weights=[0.2, 0.5, 0.3])[0]
        elif cliente['segmento'] == 'Business':
            num_conti = random.choices([1, 2, 3], weights=[0.3, 0.5, 0.2])[0]
        else:  # Retail
            num_conti = random.choices([1, 2], weights=[0.7, 0.3])[0]
        
        for i in range(num_conti):
            tipo_conto = ['Ordinario', 'Premium', 'Business', 'Junior'][
                np.random.choice([0, 1, 2, 3], p=[0.6, 0.2, 0.15, 0.05])
            ]
            
            # Saldo iniziale dipende da reddito e tipo conto
            reddito_mensile = cliente['reddito_annuo'] / 12
            saldo_base = reddito_mensile * random.uniform(0.5, 3.0)
            
            if tipo_conto == 'Premium':
                saldo_base *= random.uniform(2, 5)
            elif tipo_conto == 'Business':
                saldo_base *= random.uniform(3, 10)
            
            saldo = max(100, saldo_base + random.uniform(-500, 500))
            
            # Fido: max 3x reddito mensile
            fido = 0
            if tipo_conto in ['Ordinario', 'Premium'] and cliente['credit_score'] > 600:
                fido = reddito_mensile * random.uniform(1, 3) if random.random() < 0.3 else 0
            
            data_apertura = cliente['data_acquisizione'] if pd.notna(cliente['data_acquisizione']) else Config.DATA_INIZIO
            if isinstance(data_apertura, str):
                data_apertura = pd.to_datetime(data_apertura)
            
            conti.append({
                'conto_id': conto_id,
                'iban': genera_iban(cliente['filiale_id']),
                'cliente_id': cliente['cliente_id'],
                'tipo_conto': tipo_conto,
                'saldo_disponibile': round(saldo, 2),
                'saldo_contabile': round(saldo, 2),
                'valuta': 'EUR',
                'fido_accordato': round(fido, 2),
                'tasso_interesse': round(random.uniform(0.01, 0.5), 2),
                'data_apertura': data_apertura,
                'data_chiusura': None,
                'stato_conto': 'Attivo' if random.random() > 0.05 else random.choice(['Dormiente', 'Chiuso']),
                'filiale_id': cliente['filiale_id'],
                'created_at': datetime.now(),
                'updated_at': datetime.now()
            })
            conto_id += 1
        
        if conto_id % 10000 == 0:
            print(f"  Progresso: {conto_id} conti")
    
    df = pd.DataFrame(conti)
    print(f"✓ {len(df)} conti generati (media {len(df)/len(df_clienti):.2f} conti/cliente)")
    print(f"  Tipi: {df['tipo_conto'].value_counts().to_dict()}")
    return df

In [6]:
df_conti = genera_conti(df_clienti, df_filiali)
df_conti.to_csv(Config.DATA_DIR / "conti.csv", index=False, encoding="utf-8-sig")


Generazione conti correnti...
  Progresso: 20000 conti
  Progresso: 40000 conti
  Progresso: 60000 conti
✓ 69109 conti generati (media 1.38 conti/cliente)
  Tipi: {'Ordinario': 41450, 'Premium': 13910, 'Business': 10363, 'Junior': 3386}


##  Generazione transazioni bancarie
Caratteristiche:
- 1M+ transazioni
- Stagionalità
- Stipendi, pensioni, POS, ATM
- Flag frodi

In [13]:
def genera_transazioni_batch(df_conti: pd.DataFrame, df_clienti: pd.DataFrame, 
                             num_transazioni: int = 1000000) -> pd.DataFrame:
    """
    Genera transazioni bancarie con pattern realistici.
    Include stagionalità, stipendi ricorrenti, utenze, etc.
    """
    print(f"\nGenerazione {num_transazioni:,} transazioni...")
    
    transazioni = []
    
    # Categorie con pesi realistici
    categorie_entrate = {
        'Stipendio': 0.40,
        'Pensione': 0.25,
        'Bonifico': 0.20,
        'Altro': 0.15
    }
    
    categorie_uscite = {
        'Addebito utenze': 0.15,
        'Spesa alimentare': 0.20,
        'Carburante': 0.10,
        'Bonifico': 0.15,
        'Prelievo ATM': 0.12,
        'Pagamento POS': 0.18,
        'Ristorante': 0.05,
        'Shopping': 0.05
    }
    
    canali = ['Filiale', 'ATM', 'Online', 'Mobile', 'POS']
    canali_pesi = [0.05, 0.15, 0.35, 0.30, 0.15]
    
    tipi_trans = ['BON', 'RID', 'ATM', 'POS', 'F24', 'MAV', 'TRF']
    
    # Merge per avere info cliente nei conti
    df_conti_full = df_conti.merge(df_clienti[['cliente_id', 'reddito_annuo', 'professione']], 
                                    on='cliente_id', how='left')
    
    # Sample conti attivi con probabilità proporzionale al saldo
    conti_attivi = df_conti_full[df_conti_full['stato_conto'] == 'Attivo'].copy()
    
    for i in range(num_transazioni):
        if (i + 1) % 100000 == 0:
            print(f"  Progresso: {i+1:,}/{num_transazioni:,}")
        
        # Selezione conto con probabilità basata su saldo
        conto = conti_attivi.sample(n=1, weights='saldo_disponibile').iloc[0]
        
        # Data transazione distribuita negli anni
        giorni_range = (Config.DATA_FINE - Config.DATA_INIZIO).days
        data_trans = Config.DATA_INIZIO + timedelta(days=random.randint(0, giorni_range))
        
        # Maggiore attività nei giorni feriali
        if data_trans.weekday() >= 5:  # Weekend
            if random.random() < 0.3:  # Skip 70% weekend
                continue
        
        # Entrata o uscita
        tipo_movimento = random.choices(['Avere', 'Dare'], weights=[0.35, 0.65])[0]
        
        if tipo_movimento == 'Avere':
            categoria = np.random.choice(list(categorie_entrate.keys()), 
                                        p=list(categorie_entrate.values()))
            
            # Importi entrate
            if categoria == 'Stipendio':
                importo = conto['reddito_annuo'] / 12 * random.uniform(0.95, 1.05)
                # Stipendi a fine mese
                data_trans = data_trans.replace(day=random.randint(25, 28))
            elif categoria == 'Pensione':
                importo = conto['reddito_annuo'] / 12 * random.uniform(0.9, 1.1)
                # Pensioni primo del mese
                data_trans = data_trans.replace(day=1)
            else:
                importo = np.random.lognormal(5, 1.5)  # Media ~150€
        
        else:  # Dare (uscite)
            categoria = np.random.choice(list(categorie_uscite.keys()), 
                                        p=list(categorie_uscite.values()))
            
            # Importi uscite
            if categoria == 'Addebito utenze':
                importo = random.uniform(50, 300)
            elif categoria == 'Spesa alimentare':
                importo = random.uniform(20, 150)
            elif categoria == 'Carburante':
                importo = random.uniform(30, 80)
            elif categoria == 'Prelievo ATM':
                importo = random.choice([20, 50, 100, 150, 200])
            elif categoria == 'Pagamento POS':
                importo = np.random.lognormal(3, 1)  # Media ~20€
            else:
                importo = np.random.lognormal(4, 1.5)  # Media ~55€
        
        importo = round(max(0.01, importo), 2)
        
        # Aggiornamento saldo
        saldo_dopo = conto['saldo_disponibile']
        if tipo_movimento == 'Avere':
            saldo_dopo += importo
        else:
            saldo_dopo -= importo
        
        # Descrizione
        descrizioni = {
            'Stipendio': 'Accredito stipendio mensile',
            'Pensione': 'Accredito pensione INPS',
            'Bonifico': f'Bonifico {"da" if tipo_movimento=="Avere" else "a"} {random.choice(["Rossi", "Bianchi", "Verdi"])}',
            'Addebito utenze': random.choice(['Bolletta ENEL', 'Bolletta GAS', 'Bolletta acqua']),
            'Spesa alimentare': random.choice(['COOP', 'Lidl', 'Eurospin', 'Carrefour']),
            'Carburante': random.choice(['ENI', 'IP', 'Q8', 'Tamoil']),
            'Prelievo ATM': 'Prelievo contante',
            'Pagamento POS': 'Pagamento POS',
            'Ristorante': random.choice(['Ristorante', 'Pizzeria', 'Trattoria']),
            'Shopping': random.choice(['Amazon', 'Zalando', 'Negozio'])
        }
        descrizione = descrizioni.get(categoria, f'Operazione {categoria}')
        
        # Canale
        if categoria == 'Prelievo ATM':
            canale = 'ATM'
        elif categoria == 'Pagamento POS':
            canale = 'POS'
        elif categoria in ['Stipendio', 'Pensione']:
            canale = 'Filiale'
        else:
            canale = np.random.choice(canali, p=canali_pesi)
        
        # Tipo transazione
        if categoria == 'Prelievo ATM':
            tipo_trans_cod = 'ATM'
        elif categoria == 'Pagamento POS':
            tipo_trans_cod = 'POS'
        elif 'Bonifico' in categoria:
            tipo_trans_cod = 'BON'
        elif 'utenze' in categoria:
            tipo_trans_cod = 'RID'
        else:
            tipo_trans_cod = random.choice(tipi_trans)
        
        # Flag frode (basato su pattern anomali)
        flag_sospetta = 0
        if importo > 5000 or (importo > 1000 and canale == 'Online'):
            flag_sospetta = 1 if random.random() < Config.FRAUD_RATE else 0
        
        
        
        transazioni.append({
            'transazione_id': i + 1,
            'conto_id': conto['conto_id'],
            'tipo_transazione_id': hash(tipo_trans_cod) % 10 + 1,  # Simulato
            'data_transazione': data_trans,
            'importo': importo,
            'tipo_movimento': tipo_movimento,
            'descrizione': descrizione,
            'categoria': categoria,
            'controparte': None,
            'iban_controparte': None,
            'canale': canale,
            'saldo_dopo': round(saldo_dopo, 2),
            'stato': 'Completata' if random.random() > 0.01 else 'Pending',
            'flag_sospetta': flag_sospetta,
            'note': None,
            'created_at': datetime.now()
        })
    
    df = pd.DataFrame(transazioni)
    print(f"✓ {len(df):,} transazioni generate")
    print(f"  Entrate/Uscite: {df['tipo_movimento'].value_counts().to_dict()}")
    print(f"  Transazioni sospette: {df['flag_sospetta'].sum():,} ({df['flag_sospetta'].sum()/len(df)*100:.2f}%)")
    return df

In [14]:
df_transazioni = genera_transazioni_batch(
    df_conti, df_clienti, num_transazioni=100000
)

#  BLINDA IL TIPO PER SQL SERVER
df_transazioni["flag_sospetta"] = df_transazioni["flag_sospetta"].astype(int)

df_transazioni.to_csv(
    Config.DATA_DIR / "transazioni.csv",
    index=False,
    encoding="utf-8-sig"
)


Generazione 100,000 transazioni...
  Progresso: 100,000/100,000
✓ 91,541 transazioni generate
  Entrate/Uscite: {'Dare': 59533, 'Avere': 32008}
  Transazioni sospette: 8 (0.01%)


##  Generazione prestiti e NPL
- Prestiti personali, mutui, auto, aziendali
- Ammortamento francese
- Classificazione NPL / Past Due / Performing

In [10]:
def genera_prestiti(df_clienti: pd.DataFrame, df_filiali: pd.DataFrame, 
                    num_prestiti: int = 15000) -> pd.DataFrame:
    """
    Genera prestiti con classificazione NPL realistica.
    """
    print(f"\nGenerazione {num_prestiti:,} prestiti...")
    
    prestiti = []
    
    # Solo clienti con credit score > 550
    clienti_eligible = df_clienti[df_clienti['credit_score'] > 550].copy()
    
    tipi_prestito = {
        'Personale': {'peso': 0.40, 'importo': (5000, 30000), 'durata': (24, 84)},
        'Mutuo': {'peso': 0.30, 'importo': (50000, 300000), 'durata': (120, 360)},
        'Auto': {'peso': 0.20, 'importo': (10000, 40000), 'durata': (36, 84)},
        'Aziendale': {'peso': 0.10, 'importo': (20000, 200000), 'durata': (24, 120)}
    }
    
    finalita_mapping = {
        'Personale': ['Liquidità', 'Ristrutturazione casa', 'Spese mediche', 'Matrimonio'],
        'Mutuo': ['Acquisto prima casa', 'Acquisto seconda casa', 'Surroga mutuo'],
        'Auto': ['Acquisto auto nuova', 'Acquisto auto usata'],
        'Aziendale': ['Capitale circolante', 'Investimenti', 'Espansione']
    }
    
    for i in range(num_prestiti):
        if (i + 1) % 5000 == 0:
            print(f"  Progresso: {i+1:,}/{num_prestiti:,}")
        
        cliente = clienti_eligible.sample(n=1).iloc[0]
        
        # Tipo prestito
        tipo = np.random.choice(list(tipi_prestito.keys()), 
                               p=[v['peso'] for v in tipi_prestito.values()])
        config = tipi_prestito[tipo]
        
        # Importo e durata
        importo_erogato = round(random.uniform(*config['importo']), 2)
        durata_mesi = random.randint(*config['durata'])
        
        # Tasso interesse (dipende da credit score)
        if cliente['credit_score'] > 750:
            tan = random.uniform(3.0, 4.5)
        elif cliente['credit_score'] > 650:
            tan = random.uniform(4.5, 6.5)
        else:
            tan = random.uniform(6.5, 9.5)
        
        # Calcolo rata (ammortamento francese)
        tasso_mensile = tan / 100 / 12
        rata_mensile = importo_erogato * (tasso_mensile * (1 + tasso_mensile)**durata_mesi) / \
                      ((1 + tasso_mensile)**durata_mesi - 1)
        
        # Date
        data_erogazione = Config.DATA_INIZIO + timedelta(days=random.randint(0, 3650))
        data_scadenza = data_erogazione + timedelta(days=durata_mesi * 30)
        
        # Calcolo stato e classificazione
        mesi_trascorsi = min(durata_mesi, (datetime.now() - data_erogazione).days // 30)
        importo_residuo = importo_erogato * ((1 - (mesi_trascorsi / durata_mesi)) if mesi_trascorsi < durata_mesi else 0)
        
        # NPL ratio target: 8%
        is_npl = random.random() < Config.NPL_RATIO
        
        if is_npl:
            giorni_ritardo = random.randint(91, 365)
            classificazione = 'NPL'
            stato_prestito = 'Sofferenza'
        elif random.random() < 0.10:  # 10% past due
            giorni_ritardo = random.randint(1, 90)
            classificazione = 'Past due'
            stato_prestito = 'In corso'
        else:
            giorni_ritardo = 0
            classificazione = 'Performing'
            stato_prestito = 'In corso' if importo_residuo > 0 else 'Estinto'
        
        prestiti.append({
            'prestito_id': i + 1,
            'codice_pratica': f'PR{datetime.now().year}{str(i+1).zfill(8)}',
            'cliente_id': cliente['cliente_id'],
            'tipo_prestito': tipo,
            'importo_erogato': importo_erogato,
            'importo_residuo': round(importo_residuo, 2),
            'tasso_interesse': round(tan, 2),
            'durata_mesi': durata_mesi,
            'rata_mensile': round(rata_mensile, 2),
            'data_erogazione': data_erogazione,
            'data_scadenza': data_scadenza,
            'prossima_scadenza_rata': data_erogazione + timedelta(days=(mesi_trascorsi + 1) * 30) if stato_prestito == 'In corso' else None,
            'finalita': random.choice(finalita_mapping[tipo]),
            'garanzie': 'Ipoteca di primo grado' if tipo == 'Mutuo' else ('Pegno auto' if tipo == 'Auto' else None),
            'stato_prestito': stato_prestito,
            'classificazione': classificazione,
            'giorni_ritardo': giorni_ritardo,
            'filiale_id': cliente['filiale_id'],
            'created_at': datetime.now(),
            'updated_at': datetime.now()
        })
    
    df = pd.DataFrame(prestiti)
    print(f"✓ {len(df):,} prestiti generati")
    print(f"  Tipi: {df['tipo_prestito'].value_counts().to_dict()}")
    print(f"  Classificazione: {df['classificazione'].value_counts().to_dict()}")
    print(f"  NPL Ratio: {(df['classificazione']=='NPL').sum()/len(df)*100:.2f}%")
    return df

In [11]:
df_prestiti = genera_prestiti(df_clienti, df_filiali, num_prestiti=15_000)
df_prestiti.to_csv(Config.DATA_DIR / "prestiti.csv", index=False, encoding="utf-8-sig")


Generazione 15,000 prestiti...
  Progresso: 5,000/15,000
  Progresso: 10,000/15,000
  Progresso: 15,000/15,000
✓ 15,000 prestiti generati
  Tipi: {np.str_('Personale'): 5938, np.str_('Mutuo'): 4543, np.str_('Auto'): 3007, np.str_('Aziendale'): 1512}
  Classificazione: {'Performing': 12485, 'Past due': 1302, 'NPL': 1213}
  NPL Ratio: 8.09%
